In [34]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.models import load_model
import joblib
import tensorflow as tf
from flask import Flask, request, jsonify

In [ ]:

def search_courses(query, top_n=5):
    # Load pre-trained model
    model = load_model("model_recommender.keras")

    # Load preprocessed assets
    tfidf = pickle.load(open("tfidf_vectorizer.pkl", "rb"))  # Load the fitted vectorizer
    course_embeddings = np.load("course_embeddings.npy")
    courses = pd.read_csv("udemy_courses (1).csv")

    # Transform the query using the fitted vectorizer
    query_vec = tfidf.transform([query]).toarray()
    query_emb = model.predict(query_vec)

    # Compute similarities
    similarities = cosine_similarity(query_emb, course_embeddings).flatten()
    top_indices = similarities.argsort()[::-1][:top_n]

    return courses.iloc[top_indices].to_dict(orient="records")

In [46]:

app = Flask(__name__)

# Load assets once at startup
model = load_model("model_recommender.keras")
tfidf = pickle.load(open("tfidf_vectorizer.pkl", "rb"))
course_embeddings = np.load("course_embeddings.npy")
courses = pd.read_csv("udemy_courses (1).csv")
encoder_model = tf.keras.models.load_model('course_encoder_model.keras')

# === Define recommendation function ===
def recommend_related(course_id, courses_df, embeddings, top_n=10):
    print(type(course_id))
    if course_id not in courses_df['id'].values:
        raise ValueError(f"Course ID {course_id} not found in dataset.")

    idx = courses_df[courses_df['id'] == course_id].index[0]
    course_vec = embeddings[idx].reshape(1, -1)

    similarities = cosine_similarity(course_vec, embeddings).flatten()
    courses_df['similarity'] = similarities

    recommendations = courses_df[courses_df['id'] != course_id] \
        .sort_values(by='similarity', ascending=False) \
        .head(top_n)

    return recommendations['id'].tolist() 
    # return recommendations[['id', 'title', 'similarity']]

# === Flask route that returns JSON ===
@app.route('/recommend-by-id', methods=['POST'])
def recommend_by_id():
    data = request.get_json()
    course_id = data.get("course_id")
    course_id = int(course_id)
    if course_id is None:
        return jsonify({"error": "Missing course_id"}), 400

    try:
        recs = recommend_related(course_id, courses, course_embeddings)
        return jsonify(recs)  # Return the list directly as JSON
    except ValueError as e:
        return jsonify({"error": str(e)}), 404

@app.route('/recommend', methods=['POST'])
def recommend():
    data = request.get_json()
    query = data.get("query")

    if not query:
        return jsonify({"error": "Missing query"}), 400

    query_vec = tfidf.transform([query]).toarray()
    query_emb = model.predict(query_vec)

    similarities = cosine_similarity(query_emb, course_embeddings).flatten()
    top_indices = similarities.argsort()[::-1][:5]

    recommended_courses = courses.iloc[top_indices].to_dict(orient="records")
    return jsonify(recommended_courses)

if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [08/May/2025 01:00:52] "POST /recommend-by-id HTTP/1.1" 200 -


<class 'int'>


[2025-05-08 01:05:20,549] ERROR in app: Exception on /recommend-by-id [POST]
Traceback (most recent call last):
  File "c:\Users\moham\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 2190, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\moham\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 1486, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\moham\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 1484, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\moham\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 1469, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

<class 'int'>


...

<class 'int'>


127.0.0.1 - - [08/May/2025 01:25:23] "POST /recommend-by-id HTTP/1.1" 200 -


<class 'int'>


127.0.0.1 - - [08/May/2025 01:26:00] "POST /recommend-by-id HTTP/1.1" 200 -


<class 'int'>


In [36]:
recommend_related(221674, courses, course_embeddings, top_n=5)

755       695952
71       3792262
9486     3466516
2077      364384
51615    1910784
Name: id, dtype: int64

### return ids 


In [32]:
from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.models import load_model

app = Flask(__name__)

# Load assets once at startup
model = load_model("model_recommender.keras")
tfidf = pickle.load(open("tfidf_vectorizer.pkl", "rb"))
course_embeddings = np.load("course_embeddings.npy")
courses = pd.read_csv("udemy_courses (1).csv")

@app.route('/recommend', methods=['POST'])
def recommend():
    data = request.get_json()
    query = data.get("query")

    if not query:
        return jsonify({"error": "Missing query"}), 400

    query_vec = tfidf.transform([query]).toarray()
    query_emb = model.predict(query_vec)

    similarities = cosine_similarity(query_emb, course_embeddings).flatten()
    top_indices = similarities.argsort()[::-1][:10]

    # Get only course IDs of the recommended courses
    recommended_course_ids = courses.iloc[top_indices]['id'].tolist()

    return jsonify(recommended_course_ids)

if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step


127.0.0.1 - - [07/May/2025 20:14:43] "POST /recommend HTTP/1.1" 200 -
